In [1]:
from  Dataset import *

In [2]:
from Config import *

In [3]:
dataset = Dataset()

> Dataset Class Initialized
>> loading Data: BTCUSDT
>> Data Loaded


In [4]:
train, test = dataset.ProcessData()

In [5]:
train

,Low,High,Open,Close,Volume,buy,sell
time,,,,,,,
1590949860000,0.107939,0.107586,0.107577,0.107774,0.004642,0,0
1590949920000,0.107903,0.107516,0.107587,0.107738,0.003763,0,0
1590949980000,0.107699,0.107464,0.107550,0.107519,0.010451,0,0
1590950040000,0.107700,0.107321,0.107333,0.107564,0.006700,0,0
1590950100000,0.107697,0.107313,0.107384,0.107544,0.006557,0,0
...,...,...,...,...,...,...,...
1609439100000,0.418519,0.417805,0.417842,0.418155,0.009330,0,0
1609439160000,0.418846,0.417849,0.418025,0.418158,0.005514,0,0
1609439220000,0.418884,0.417859,0.418036,0.418109,0.008732,0,0


In [ ]:
def make_seq(df):
    seq_data = []
    deq = deque(maxlen=LOOK_BACK_LEN)
    for index, row in df.iterrows():
        deq.append(row[:len(df.columns)-2].values)
#         print(row[:len(df.columns)-2].values)
        if len(deq) == LOOK_BACK_LEN:
#             print(row[len(df.columns)-2:].values)
            seq_data.append([ np.array(list(deq)),  np.array(row[len(df.columns)-2:].values) ])
    return seq_data

In [ ]:
def split_xy(arr):
    x = []
    y = []

    for seq, target in train_seq:
        x.append(seq)
        y.append(target)

    x = np.array(x)
    y = np.array(y)
    return x,y

In [ ]:
train_seq  = make_seq(train)


In [ ]:
print(train_seq[0])

In [ ]:
print(x.shape)
print(y.shape)

In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras import Sequential
import tensorflow as tf

In [10]:
ts_generator = TimeseriesGenerator(train[[*train.columns[:len(train.columns)-2]]].values, train[[*train.columns[-1:]]].values, length=LOOK_BACK_LEN, sampling_rate=1, batch_size=50)

5

In [16]:
model = Sequential()
model.add(LSTM(128, input_shape=(360,len(ts_generator[0][0][0][0])), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [17]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
model.fit(
    ts_generator,
    epochs=20,
    shuffle=False
)
